In [100]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow as tf
import matplotlib.pyplot as plt
import glob
from PIL import Image
import numpy as np
import pandas as pd
import time
from sklearn import preprocessing

In [101]:
# read files from directory https://www.tensorflow.org/tutorials/images/classification
# it uses generators to reduce the memory
desire_size = (100,100)
batch_size = 512
train_ds = tf.keras.utils.image_dataset_from_directory(
  "data/train",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=desire_size,
  batch_size=batch_size,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  "data/train",
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=desire_size,
  batch_size=batch_size,
)

class_names = train_ds.class_names

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 4252 files belonging to 11 classes.
Using 3402 files for training.
Found 4252 files belonging to 11 classes.
Using 850 files for validation.


In [102]:
data_augmentation = Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.4),
  tf.keras.layers.RandomZoom(0.4), 
  tf.keras.layers.RandomContrast(0.4),
  # tf.keras.layers.RandomWidth(0.2, interpolation="nearest")
  # tf.keras.layers.RandomTranslation(0.4,0.4)
  # tf.keras.layers.RandomBrightness(0.1)
])

In [103]:
# network using all the channels
model2 = Sequential([
  layers.Rescaling(1./255, input_shape=(desire_size[0], desire_size[1], 3)),
  data_augmentation,
  layers.Conv2D(16, 5, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 5, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 5, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.1),
  layers.Conv2D(128, 5, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(256, 7, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.4),
  layers.Flatten(),
  layers.Dense(216, activation='relu'),
  layers.Dense(len(class_names))
])
model2.compile(optimizer='adam',
               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

In [104]:
epochs = 15
history = model2.fit(
  train_ds,
  validation_data = val_ds,
  epochs = epochs
) 

Epoch 1/15
7/7 [==============================] - 98s 11s/step - loss: 2.4026 - accuracy: 0.1405 - val_loss: 2.3918 - val_accuracy: 0.1282
Epoch 2/15
7/7 [==============================] - 78s 11s/step - loss: 2.3819 - accuracy: 0.1343 - val_loss: 2.3698 - val_accuracy: 0.1282
Epoch 3/15
7/7 [==============================] - 71s 10s/step - loss: 2.3561 - accuracy: 0.1343 - val_loss: 2.3507 - val_accuracy: 0.1282
Epoch 4/15
7/7 [==============================] - 71s 10s/step - loss: 2.3389 - accuracy: 0.1382 - val_loss: 2.3380 - val_accuracy: 0.1447
Epoch 5/15
7/7 [==============================] - 75s 11s/step - loss: 2.3292 - accuracy: 0.1461 - val_loss: 2.3207 - val_accuracy: 0.1447
Epoch 6/15
7/7 [==============================] - 78s 11s/step - loss: 2.3098 - accuracy: 0.1461 - val_loss: 2.3444 - val_accuracy: 0.1459
Epoch 7/15
7/7 [==============================] - 73s 11s/step - loss: 2.2989 - accuracy: 0.1626 - val_loss: 2.2592 - val_accuracy: 0.1506
Epoch 8/15
7/7 [===========

KeyboardInterrupt: 

In [ ]:
time.sleep(15)
epochs = 15 
history = model2.fit(
  train_ds,
  validation_data = val_ds,
  epochs = epochs
)

In [ ]:
time.sleep(15)
epochs = 15
history = model2.fit(
  train_ds,
  validation_data = val_ds,
  epochs = epochs
)

In [ ]:
time.sleep(15)
epochs = 15
history = model2.fit(
  train_ds,
  validation_data = val_ds,
  epochs = epochs
)

In [ ]:
time.sleep(15)
epochs = 15
history = model2.fit(
  train_ds,
  validation_data = val_ds,
  epochs = epochs
)

In [ ]:
time.sleep(15)
epochs = 15
history = model2.fit(
  train_ds,
  validation_data = val_ds,
  epochs = epochs
)

In [ ]:
time.sleep(15)
epochs = 15
history = model2.fit(
  train_ds,
  validation_data = val_ds,
  epochs = epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
array_image = []
array_name = []
for file in glob.glob("data/test/*"):
  img = tf.keras.utils.load_img(
    file, 
    target_size=desire_size
  )
  img_array = tf.keras.utils.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)
  predict = model2.predict(img_array)
  score = tf.nn.softmax(predict[0])
  array_image.append(class_names[np.argmax(score)])
  array_name.append(file.split("\\")[1])

array_image

In [ ]:
df_oob = pd.DataFrame(data= zip(array_name, array_image),
                      columns=["file","species"])
df_oob.to_csv("data/prediction_cnn.csv", index=False)

In [ ]:
model2.save('model_2_gray_form_first') 